In [1]:
# Create concepts using wikipedia (and its knowledge graph) 
# https://en.wikipedia.org/wiki/List_of_sports

# Search of sports: Found a link "List of sports"

In [1]:
import pandas as pd
import pyarrow as pa
from datasets import load_from_disk, Dataset

In [2]:
path_dir = '/work3/s174498/concept_random_dataset/wikipedia_20220301/train'
wikipedia = load_from_disk(path_dir)

In [3]:
wikipedia

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 6458670
})

In [4]:
# Physical sports

titles = wikipedia['title']


In [5]:

list_of_sports = ['Acrobatic sports',
'Air sports',
'Archery',
'Ball games',
'Ball-over-net games',
'Basketball family',
'Bat-and-ball',
'Board sports',
'Catching games',
'Climbing',
'#Cycling',
'Bicycle',
'Skibob',
'Unicycle',
'#Combat sports: wrestling and martial arts',
'Grappling',
'Striking',
'Mixed or hybrid',
'Weapons',
'Other',
'Cue sports',
'Dog sports',
'Equestrian sports',
'Fishing',
'Flying disc sports',
'Football',
'Golf',
'Gymnastics',
'Competitive yoga',
'Handball family',
'Hunting',
'Ice sports',
'Kite sports',
'Mixed discipline',
'Orienteering family',
'Pilota family',
'Parkour/Freerunning',
'#Racket (or racquet) sports',
'Sports that use a netted racket',
'Sports that use a non-netted racket, or paddle',
'Remote control',
'Rodeo-originated',
'Running',
'Sailing',
'#Snow sports',
'Skiing',
'Sled sports',
'#Shooting sports',
'Individual',
'Athletic hybrid',
'Skirmish',
'Stacking',
'#Stick and ball games',
'Hockey',
'Hurling and shinty',
'Lacrosse',
'Polo',
'Street sports',
'Strength sports',
'Tag games',
'Walking',
'Wall-and-ball',
'#Aquatic and paddle sports',
'Canoeing',
'Kayaking',
'Rafting',
'Rowing',
'Other paddling sports',
'#Aquatic ball sports',
'Competitive swimming',
'Subsurface and recreational',
'Diving',
'Weightlifting',
'#Motorized sports',
'Auto racing',
'Motorboat racing',
'Motorcycle racing',
'ATV racing',
'Marker sports',
'Overlapping sports',
'Other',
'Speedcubing']

In [11]:
list_of_gender = [
'Agender',
'Androgyne',
'Androgynous',
'Bakla',
'Bigender',
'Binary',
'Bissu',
'Butch',
'Calabai',
'Calalai',
'Cisgender',
'Demiflux',
'Demigender',
'Dual gender',
'Endosex',
'Female',
'Femme',
'Gender bender',
'Gender diverse',
'Genderfluid',
'Genderflux',
'Genderfuck',
'Genderless',
'Gender nonconforming',
'Genderqueer',
'Gender variant',
'Intergender',
'Intersex',
'Hijra',
'Kathoey',
'Male',
'Man',
'Multigender',
'Muxe',
'Neurogender',
'Neutrois',
'Non-binary',
'Omnigender',
'Pangender',
'Polygender',
'Third gender',
'Trans man',
'Trans woman',
'Transgender',
'Transfeminine',
'Transmasculine',
'Transsexual',
'Travesti',
'Trigender',
'Two spirit',
'Woman',
'X-gender',
'Xenogender']

In [12]:
list_of_interest = list_of_gender

count_datasets = 0
concept_list = []
for i in range(len(list_of_interest)): 
    if list_of_interest[i] in titles:

        idx = titles.index(list_of_interest[i])

        # Clean up data (\n references in wiki article)
        text_clean = wikipedia[idx]["text"].replace('\n\n', ' ')
        text_clean2 = text_clean.replace('\n', ' ')
        phrase_to_list = text_clean2.split(". ")

        if len(phrase_to_list) > 150:
            count_datasets = count_datasets + 1
            concept_list.append(list_of_interest[i])

            df = pd.DataFrame({'id': wikipedia[idx]["id"], 'url': wikipedia[idx]["url"], 
            'title': wikipedia[idx]["title"], 'text': wikipedia[idx]["text"], 'text_list': phrase_to_list})
            data_concept = Dataset(pa.Table.from_pandas(df))

            # save data
            data_concept.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/gender_concepts/' + list_of_interest[i])

In [13]:
print(count_datasets)
print(len(list_of_sports))
concept_list

6
82


['Intersex', 'Kathoey', 'Third gender', 'Transgender', 'Transsexual', 'Woman']

In [3]:
path_dir = '/work3/s174498/concept_random_dataset/wikipedia_20220301/concepts/Cycling'
wikipedia = load_from_disk(path_dir)